In [641]:
import requests
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm_notebook

# Tinyshakespeare dataset

In [540]:
url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
res = requests.get(url)
res.raise_for_status()
text = res.content.decode("utf-8")

In [541]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [542]:
print(f'length of dataset in characters: {len(text)}')

length of dataset in characters: 1115394


In [543]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f"{''.join(chars)} \nNumber of token: {vocab_size}")


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz 
Number of token: 65


In [544]:
# text tokenization in character level
itos = {i:c for i, c in enumerate(chars)}
stoi = {c:i for i, c in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s] # encoder: string to list of int
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: list of int to string

In [545]:
print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [48]:
# Other word tokenize
# https://github.com/google/sentencepiece
# https://github.com/openai/tiktoken

In [52]:
# trade-off code book size and seq length
# long seq of int with small vocab 
# short seq of int with large vocab

In [546]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


# Bigram model

In [547]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [548]:
# Train Test split
n = int(0.9*len(data))
train = data[:n]
test = data[n:]

In [552]:
block_size = 8 # context length
train[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [555]:
# time dimension
x = train[:block_size]
y = train[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"Input: {context} | target: {target}")

Input: tensor([18]) | target: 47
Input: tensor([18, 47]) | target: 56
Input: tensor([18, 47, 56]) | target: 57
Input: tensor([18, 47, 56, 57]) | target: 58
Input: tensor([18, 47, 56, 57, 58]) | target: 1
Input: tensor([18, 47, 56, 57, 58,  1]) | target: 15
Input: tensor([18, 47, 56, 57, 58,  1, 15]) | target: 47
Input: tensor([18, 47, 56, 57, 58,  1, 15, 47]) | target: 58


In [554]:
# batch dimension
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train if split == 'train' else test
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

xb, yb = get_batch('train')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"Input: {context} | target: {target}")
    print("---"*20)

Input: tensor([24]) | target: 43
Input: tensor([24, 43]) | target: 58
Input: tensor([24, 43, 58]) | target: 5
Input: tensor([24, 43, 58,  5]) | target: 57
Input: tensor([24, 43, 58,  5, 57]) | target: 1
Input: tensor([24, 43, 58,  5, 57,  1]) | target: 46
Input: tensor([24, 43, 58,  5, 57,  1, 46]) | target: 43
Input: tensor([24, 43, 58,  5, 57,  1, 46, 43]) | target: 39
------------------------------------------------------------
Input: tensor([44]) | target: 53
Input: tensor([44, 53]) | target: 56
Input: tensor([44, 53, 56]) | target: 1
Input: tensor([44, 53, 56,  1]) | target: 58
Input: tensor([44, 53, 56,  1, 58]) | target: 46
Input: tensor([44, 53, 56,  1, 58, 46]) | target: 39
Input: tensor([44, 53, 56,  1, 58, 46, 39]) | target: 58
Input: tensor([44, 53, 56,  1, 58, 46, 39, 58]) | target: 1
------------------------------------------------------------
Input: tensor([52]) | target: 58
Input: tensor([52, 58]) | target: 1
Input: tensor([52, 58,  1]) | target: 58
Input: tensor([52, 5

In [642]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size, n_embd):
        super().__init__()
        self.token_embbeding_table = nn.Embedding(vocab_size, n_embd)
                                                          
    def forward(self, idx, targets=None):
        logits = self.token_embbeding_table(idx) # (B, T, C) batch, time=context length, channel=n_embd
        if targets is None:
            loss = None
        else:
            # change logits shape for cross entropy loss (B, C)
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # (B, C)
            probs = F.softmax(logits, dim=-1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.concat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [643]:
model = BigramLanguageModel(vocab_size, vocab_size).to(device)

In [644]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
sum(p.numel() for p in model.parameters())
epochs = 10000

In [645]:
batch_size = 32
for epoch in tqdm_notebook(range(epochs)):
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

C:\Users\bJ5xp\AppData\Local\Temp\ipykernel_22788\1138750604.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for epoch in tqdm_notebook(range(epochs)):


  0%|          | 0/10000 [00:00<?, ?it/s]

2.4938886165618896


In [646]:
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), 500)[0].tolist()))


&qFbTzXxuNJ
mBH3c'v:ykYxIaYqKzlYyiLyguuJ:shdaFY
WJjbzB'tAq.zX-B-ISgwy-BGM,ebpkNePEWR!ElCQXYv!vQZZ!FDdZ
izNHfLKrynoOYEyk.wGR!YBZ,VG-iRK;HxLEXYZQo.w cTJPxgzz:lCorVwy:!nwHltSQZwyeuSQ:fwZBEyaAPUzJywkRrW;ZErYyw$3qFc;wylay'Xx
K;&;cc;HFltgMXJ!p
IgTX-VOS,:,IQXzt.,D aY,
VDD?mNg
iEDDDszfbzihNXZ,hRid:ZVwgw d'vJsuxKhbD,I:dCqFSlaOPatJRN&P'Olwyh$ThpTS!D,N?;XPfPPv:$BhaD,NXQ.VRqL:b?OC.wpivH,yxKQCg?'Xm:!:lscp3gwpir ZckiuyPidBLhrZZHfXtPxaAh3;dIQH.
d;vJldg3BGuxri.bF?XV!vJ!?YjTHf -A?
CfQ&Pmned:VPsZG$''vPakR-'TXYnm3


# The math trick in self-attention

# <----------------------------------------------

In [418]:
# self-attention
torch.manual_seed(1337)
B, T, C = 4, 8 ,32 # batch, time, channel
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 32])

In [419]:
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros(T, T)# init affinities between tokens
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=-1)
out = wei @ x # (T, T) @ (B, T, C) = (B, T, C)
out.shape

torch.Size([4, 8, 32])

# V2

In [352]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size, n_embd):
        super().__init__()
        self.token_embbeding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embbeding_table = nn.Embedding(ctx_len, n_embd)
        self.sd_heads = MultiHeadAttention(4, n_embd//4)
        self.lm_head = nn.Linear(n_embd, vocab_size)
    
    def forward(self, idx, targets=None):
        B, T = idx.shape
        token_emb = self.token_embbeding_table(idx) # (B, T, C)  C=n_embd
        pos_emb = self.position_embbeding_table(torch.arange(T, device=device)) # (T, C) C=n_embd
        x = token_emb + pos_emb
        x = self.sd_heads(x)
        logits= self.lm_head(x) # (B, T, C) C=vocab_size
        
        if targets is None:
            loss = None
        else:
            # change logits shape for cross entropy loss (B, C)
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -ctx_len:]
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # (B, C)
            probs = F.softmax(logits, dim=-1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.concat((idx, idx_next), dim=1) # (B, T+1)
        return idx

# Single Self Attention Head

In [522]:
# self-attention
torch.manual_seed(1337)
B, T, C = 4, 8 ,32 # batch, time, channel
x = torch.randn(B, T, C) 

# single head - self-attention (decoder block)
head_size = 16
query = nn.Linear(C, head_size, bias=False) # (B,T,C) @ (C, H) -> (B,T,H)
key = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
q = query(x) # (B,T,H)
k = query(x) # (B,T,H)
wei = q @ k.transpose(-2, -1) / head_size**0.5# (B, T, H) @ (B, H, T) = (B, T, T) 

tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril==0, float('-inf')) # masked head attention
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v

out.shape

torch.Size([4, 8, 16])

In [529]:
# single head self-attention
class Head(nn.Module):
    def __init__(self, n_embd, head_size, context_length):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(context_length, context_length)))
        
    def forward(self, x):
        B, T, C = x.shape # (B, T, C) C=n_embd
        k = self.key(x) # (B, T, C) @ (C, H) = (B, T, H)
        q = self.query(x) # (B, T, C) @ (C, H) = (B, T, H)
        # attention scores (affinities)
        att = q @ k.transpose(-2, -1) / head_size**0.5
        att = att.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        att = F.softmax(att, dim=-1)
        # weighted sum of the values
        v = self.value(x) # (B, T, C) @ (C, H) = (B, T, H)
        out = att @ v
        return out
        

# Multi-Head Self Attention

In [530]:
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        
    def forward(self, x):
        return torch.cat([h(x) for h in self.heads], dim=-1)